In [ ]:
# Import all dependencies
import cv2 
import os
import shutil
import numpy as np
from matplotlib import pyplot as plt
%matplotlib qt
from tensorflow.keras.models import load_model
import tensorflow as tf
import tkinter as tk
from tkinter import simpledialog, Button
from collections import OrderedDict
import math
from natsort import natsorted

In [ ]:
# Graphical acceleration
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
# Active local webcam
camera = cv2.VideoCapture(0)

In [ ]:
# Functions for data labeling
input_value = ''
def click_Pain():
    global input_value
    input_value = f'Pain'
    window.destroy()
    
def click_Neutral():
    global input_value
    input_value = f'Neutral'
    window.destroy()

In [ ]:
# Init base variable 
# List to store images from Image Catcher
images = []
image_names = []
# Get current directory path
current_dir = os.getcwd()
# Check if 'test_iamges' folder exists, if not create folder
if 'test_images' not in os.listdir(path=current_dir):
    os.mkdir('test_images')
    file_number = 0
else:
    files = os.listdir('test_images')
    # If there is no file, set file_number as 0, otherwise sort list as Windows, and get the number
    if not files:
        file_number = 0
    else:
        file_to_check = natsorted(files)[-1]
        file_number = int(file_to_check[0:file_to_check.index('_')]) + 1

# Image capture loop
while True:
    # Get video frame
    _, camera_frame = camera.read()
    # Draw rectangle on catched image - show area of proper photo
    image_with_border = cv2.rectangle(camera_frame, (100, 100), (500, 500), (255,0,0), 2)
    # Show image with drawn border
    cv2.imshow("Image Catcher", image_with_border)
    # Wait for 1[s] to take an action
    pressed_key = cv2.waitKey(1)
    # If 'q' button clicked, exit "Image Catcher"
    if pressed_key == 113:
        print('Exited camera window')
        # Close all opend Windows
        cv2.destroyAllWindows()
        break 
    # If 'space' button clicked, take an image
    if pressed_key == 32:
        # Get clear frame (without border)
        _, image = camera.read()
        # Append list with taken image of size 500x500
        images.append(image[100:500, 100:500])         
        
        # Define window for manual labeling
        window = tk.Tk()        
        # Define two push buttons with labeling names   
        button_v1 = Button(window, text = "Pain", command=click_Pain, width = 100,)
        button_v2 = Button(window, text = "Neutral", command=click_Neutral,  width = 100)
        button_v1.pack(ipadx=10, ipady=10)
        button_v2.pack(ipadx=10, ipady=10)
        window.mainloop()  
        
        # Set path to save taken image
        image_names.append(f'{file_number}_{input_value}.jpg')
        # Save image 
        cv2.imwrite(f'test_images/{image_names[-1]}', image[100:500, 100:500])
        # Print information about saved image
        print(f"Image saved to list: {file_number}") 
        
        # Iterate file name by 1
        file_number += 1
       

In [ ]:
# Release used device - built-in camera
camera.release()

In [ ]:
# Reading images from directory
if not images:
    file = 'test_images'
    images = []
    image_names = os.listdir(file) # Images name read from file

    for img_name in image_names:
        img = cv2.imread(f'{file}/{img_name}')     
        images.append(img)

In [ ]:
predict_emotion = []
predicted_values = []
# Load pretrain model of CNN
model = load_model(os.path.join('models', 'pain_detection_model_VGG16.h5')) # Change pain_detection_modelVGG16.h5 for diffrent model
# Loop through images list
for index, img in enumerate(images):
    # Resize image to 256x256 size -> expected size by CNN model
    resized_img = tf.image.resize(img, (224,224))
    # Predict pain image
    predict_emotion.append(model.predict(np.expand_dims(resized_img/255, 0)))
    predicted_values.append(predict_emotion[index][0][0])
    
# Waiting for key action
cv2.waitKey(0)
# Closing all opend windows with images
cv2.destroyAllWindows()


In [ ]:
# Print images taken with camera with the wages of pain confidance
fig = plt.figure()
for i in range(0, len(images)):
    plt.subplot(math.ceil(len(images)/5), 5, i+1)
    plt.imshow(cv2.cvtColor(images[i], cv2.COLOR_BGR2RGB))
    plt.gca().set_title('%s: %.3f' %(image_names[i], predicted_values[i]), fontsize=10)    
fig.tight_layout()
fig.suptitle('Test cases images', fontsize=15)
plt.show()

In [ ]:
# Print scatter plot presenting results - false classification - red, proper classification - green 
fig_1 = plt.figure()
for index, value in enumerate(predicted_values):
    if (image_names[index][-11:-4] == ("Neutral" or "_Neutral") and value < 0.5) or (image_names[index][-8:-4] == ("Pain" or "_Pain") and value > 0.5):    
        plt.scatter(index, np.array(value),  color='green', marker ='s', label='Correct Value')
    else:
        plt.scatter(index, np.array(value),  color='red', marker ='s', label='False Value')
plt.plot(np.arange(0, len(predicted_values),1, dtype = int), [0.5 for i in range(len(predicted_values))], color = 'teal', label='Dividing line at 0.5', linewidth=2)
fig_1.suptitle('Pain prediction values', fontsize=15)

# Print unique labeles value - depends on data classfication
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())
plt.xlabel('Images', fontsize=10)
plt.ylabel('Wage', fontsize=10)
plt.show()

In [ ]:
# Delete 'test_images' directory after finshed work - optional to use
current_dir = os.getcwd()
shutil.rmtree(os.path.join(current_dir, 'test_images'))
